In [1]:
import sys
from pathlib import Path

In [2]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

2022-11-02 16:23:21.160860: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-02 16:23:21.295104: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-02 16:23:21.793238: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64
2022-11-02 16:23:21.793310: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: can

In [3]:
sys.path.append(str(Path("../../ssl").resolve()))

In [4]:
from src.models.pi_model.pi_model import PiModel
from src.models.pi_model.pi_model_config import PiModelConfig
from src.trainers.pi_model.pi_model import PiModelTrainer
from src.trainers.pi_model.pi_model_config import PiModelTrainerConfig
from src.data_loaders.pi_model.pi_model_config import PiModelDataLoaderConfig
from src.data_loaders.pi_model.pi_model import PiModelDataLoader

# Description

In this notebook, the AlexNet model will be trained on the CIFAR-10 dataset using only 25% of the labelled dataset.

## Set up Experiment

In [5]:
class TrainerConfig(PiModelTrainerConfig):
    num_epochs = 150
    loss_ramp_up_epochs = 100
    unsup_loss_weight = 5.0

train_config = TrainerConfig()

In [6]:
class ModelConfig(PiModelConfig):
    input_shape = (32, 32, 3)
    output_shape = 10

model_config = ModelConfig()

In [7]:
class DataLoaderConfig(PiModelDataLoaderConfig):
    batch_size = 64
    num_classes = 10

    blur_params = {
        'chance': 0.10,
        'kernel_ratio': 0.10,
        'blur_strength': (0.1, 2.0)
    }

    crop_params = {
        'chance': 0.50,
        'crop_size': (0.08, 1.0),
        'aspect_range': (0.75, 1.33),
        'num_tries': 100
    }

    jitter_params = {
        'chance': 0.50,
        'distort_strength': 0.20,
        'drop_chance': 0.05
    }

data_loader_config = DataLoaderConfig()

## Get Datasets

In [8]:
(x_train_full, y_train_full), (x_test_full, y_test_full) = tf.keras.datasets.cifar10.load_data()

In [9]:
# get train dataset by subsampling 10% of the full training dataset (stratified by labels)
# add the rest as unlabelled samples
x_train_unlabelled, x_train_labelled, y_train_unlabelled_, y_train_labelled = train_test_split(
    x_train_full, y_train_full,
    stratify = y_train_full,
    test_size = 0.20, random_state = 42
)
y_train_unlabelled = (-1 * np.ones_like(y_train_unlabelled_)).astype(np.uint8)

In [10]:
x_train = np.concatenate((x_train_unlabelled, x_train_labelled), axis = 0)
y_train = np.concatenate((y_train_unlabelled, y_train_labelled), axis = 0)

In [11]:
x_train, y_train = shuffle(x_train, y_train, random_state = 42)

In [12]:
# train_data_unlabelled = tf.data.Dataset.from_tensor_slices((x_train_unlabelled, y_train_unlabelled))
# train_data_labelled = tf.data.Dataset.from_tensor_slices((x_train_labelled, y_train_labelled))

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_data = tf.data.Dataset.from_tensor_slices((x_test_full, y_test_full))

2022-11-02 16:23:23.990233: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-02 16:23:23.996984: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-02 16:23:23.997538: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-02 16:23:23.998356: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [13]:
# create train dataset
train_data = PiModelDataLoader(train_data, data_loader_config)(training = True)

In [14]:
# create test dataset
val_data = PiModelDataLoader(val_data, data_loader_config)(training = False)

In [15]:
print(f"Train dataset size: {train_data.cardinality()}")
print(f"Validation dataset size: {val_data.cardinality()}")

Train dataset size: 782
Validation dataset size: 157


In [16]:
# for (x1,x2,x3,x4,y) in train_data.take(1):
#     print(x1, x2, x3, x4, y)

# Train Model

In [17]:
model = PiModel(model_config)()

In [18]:
trainer = PiModelTrainer(
    model, train_data, train_config,
    val_dataset = val_data)

In [19]:
trainer.train()

2022-11-02 16:23:27.066445: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204


Training loss at epoch 0 is : 2.08. Validation loss is : 1.87. Validation acc. is : 27.50.
Training loss at epoch 1 is : 1.92. Validation loss is : 1.71. Validation acc. is : 35.14.
Training loss at epoch 2 is : 1.82. Validation loss is : 1.67. Validation acc. is : 37.69.
Training loss at epoch 3 is : 1.76. Validation loss is : 1.64. Validation acc. is : 39.90.
Training loss at epoch 4 is : 1.70. Validation loss is : 1.55. Validation acc. is : 43.81.
Training loss at epoch 5 is : 1.66. Validation loss is : 1.47. Validation acc. is : 46.31.
Training loss at epoch 6 is : 1.61. Validation loss is : 1.41. Validation acc. is : 48.90.
Training loss at epoch 7 is : 1.57. Validation loss is : 1.37. Validation acc. is : 49.28.
Training loss at epoch 8 is : 1.54. Validation loss is : 1.36. Validation acc. is : 49.95.
Training loss at epoch 9 is : 1.50. Validation loss is : 1.31. Validation acc. is : 53.29.
Training loss at epoch 10 is : 1.46. Validation loss is : 1.28. Validation acc. is : 53.74